In [1]:
import numpy as np 

In [2]:
with open('1268-0.txt','r',encoding='utf-8') as fp:     #檔案讀取
    text = fp.read()

In [3]:
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx =text.find('End of the Project Gutenberg')
text = text[start_indx:end_indx]                     #文章 頭->尾
char_set = set(text)                                 #唯一字符
print('length:{}\nunique:{}'.format(len(text),len(char_set)))

length:1130710
unique:85


In [4]:
chars_sorted = sorted(char_set)   #排序
char2int = {ch:i for i,ch in enumerate(chars_sorted)}   #字元對應整數 0~84
char_array = np.array(chars_sorted)                     #透過np最索引
text_encoded = np.array(                                #透過np最索引
    [char2int[ch] for ch in text],dtype = np.int32)
print(text[0:20],'<<encoding>>',text_encoded[0:20])
print(text_encoded[0:20],'==reverse==',''.join(char_array[text_encoded[0:20]]))
print('對應整數:\n',char2int)

THE MYSTERIOUS ISLAN <<encoding>> [48 36 33  1 41 53 47 48 33 46 37 43 49 47  1 37 47 40 29 42]
[48 36 33  1 41 53 47 48 33 46 37 43 49 47  1 37 47 40 29 42] ==reverse== THE MYSTERIOUS ISLAN
對應整數:
 {'\n': 0, ' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '=': 27, '?': 28, 'A': 29, 'B': 30, 'C': 31, 'D': 32, 'E': 33, 'F': 34, 'G': 35, 'H': 36, 'I': 37, 'J': 38, 'K': 39, 'L': 40, 'M': 41, 'N': 42, 'O': 43, 'P': 44, 'Q': 45, 'R': 46, 'S': 47, 'T': 48, 'U': 49, 'V': 50, 'W': 51, 'X': 52, 'Y': 53, 'Z': 54, 'a': 55, 'b': 56, 'c': 57, 'd': 58, 'e': 59, 'f': 60, 'g': 61, 'h': 62, 'i': 63, 'j': 64, 'k': 65, 'l': 66, 'm': 67, 'n': 68, 'o': 69, 'p': 70, 'q': 71, 'r': 72, 's': 73, 't': 74, 'u': 75, 'v': 76, 'w': 77, 'x': 78, 'y': 79, 'z': 80, '‘': 81, '’': 82, '“': 83, '”': 84}


In [5]:
import tensorflow as tf
ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)#建立tf數據集
for ex in ds_text_encoded.take(20):
    print('{}==>{}'.format(ex.numpy(),char_array[ex.numpy()])) #1為空白鍵
print(ds_text_encoded)                  

48==>T
36==>H
33==>E
1==> 
41==>M
53==>Y
47==>S
48==>T
33==>E
46==>R
37==>I
43==>O
49==>U
47==>S
1==> 
37==>I
47==>S
40==>L
29==>A
42==>N
<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>


In [6]:
chunk_size = 41
ds_chunks = ds_text_encoded.batch(chunk_size,drop_remainder=True) #最後一批少於41字元就刪除

def spilt_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq,target_seq

ds_sequences = ds_chunks.map(spilt_input_target)

for example in ds_sequences.take(3):
    print('inp (x):',repr(''.join(char_array[example[0].numpy()])))
    print('inp (x):',repr(''.join(char_array[example[1].numpy()])))

inp (x): 'THE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTER'
inp (x): 'HE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTERI'
inp (x): 'OUS ISLAND\n\nby Jules Verne\n\n1874\n\n\n\n\nPAR'
inp (x): 'US ISLAND\n\nby Jules Verne\n\n1874\n\n\n\n\nPART'
inp (x): ' 1--DROPPED FROM THE CLOUDS\n\n\n\nChapter 1'
inp (x): '1--DROPPED FROM THE CLOUDS\n\n\n\nChapter 1\n'


In [7]:
batch_size = 64
buffer_size = 10000
ds = ds_sequences.shuffle(buffer_size).batch(batch_size)

# 建立RNN模型

In [13]:
#def build_model(vocab_size,embedding_dim,rnn_units):
#    model = tf.keras.Sequential([
#        tf.keras.layers.Embedding(vocab_size,embedding_dim),
#        tf.keras.layers.LSTM(
#            rnn_units,
#            return_sequences=True),
#        tf.keras.layers.Dense(vocab_size)
#    ])
#    return model
#
#charset_size = len(char_array)
#embedding_dim = 256 
#rnn_units = 512
#
#model = build_model(charset_size,embedding_dim,rnn_units)
#model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         21760     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1574912   
                                                                 
 dense (Dense)               (None, None, 85)          43605     
                                                                 
Total params: 1,640,277
Trainable params: 1,640,277
Non-trainable params: 0
_________________________________________________________________


# 訓練模型

In [14]:
#model.compile(
#    optimizer='adam',\
#    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
#model.fit(ds,epochs=20)

Epoch 1/20
431/431 [==============================] - 120s 275ms/step - loss: 2.3061
Epoch 2/20
431/431 [==============================] - 119s 275ms/step - loss: 1.7417
Epoch 3/20
431/431 [==============================] - 118s 274ms/step - loss: 1.5352
Epoch 4/20
431/431 [==============================] - 119s 274ms/step - loss: 1.4178
Epoch 5/20
431/431 [==============================] - 118s 273ms/step - loss: 1.3455
Epoch 6/20
431/431 [==============================] - 118s 272ms/step - loss: 1.2948
Epoch 7/20
431/431 [==============================] - 118s 273ms/step - loss: 1.2565
Epoch 8/20
431/431 [==============================] - 118s 273ms/step - loss: 1.2269
Epoch 9/20
431/431 [==============================] - 118s 272ms/step - loss: 1.2013
Epoch 10/20
431/431 [==============================] - 118s 272ms/step - loss: 1.1795
Epoch 11/20
431/431 [==============================] - 118s 273ms/step - loss: 1.1600
Epoch 12/20
431/431 [==============================] - 118s 273

In [8]:
#model.save('rnn_lage.h5')
from tensorflow.keras.models import load_model

# 產生 新文本

In [9]:
def sample(model,starting_str,len_str=200,max_input=40,scale_factor=2.0):  #1模組>2輸入字串>3文章長度>4最大輸入值>5正規化
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input,(1,-1))
    generater = starting_str
    model.reset_states()
    for i in range(len_str):
        logits = model(encoded_input)
        logits = tf.squeeze(logits,0)
        
        scaled_logits = logits*scale_factor
        new_char_indx = tf.random.categorical(scaled_logits,num_samples=1)
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()
        
        generater += str(char_array[new_char_indx])
        
        new_char_indx = tf.expand_dims([new_char_indx],0)
        encoded_input = tf.concat([encoded_input,new_char_indx],axis = 1)
        encoded_input = encoded_input[:,-max_input:]
    return generater

In [11]:
model = load_model('rnn_lage.h5')
print(sample(model,starting_str='i live in Taiwan'))

i live in Taiwan into the channel, and the sailor was so best to ascertain a distance of a considerable cast, which was the settlers would have been to the corral. The mountain was the result of the island, which was
